In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import pandas as pd
from tqdm import tqdm_notebook
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
from functools import reduce
plt.rcParams['figure.dpi'] = 120
import h5py
from glob import glob
import re

if '../scripts' not in sys.path:
    sys.path.append('../scripts')
from importlib import reload
import figure_template
# force reload of the module
reload(figure_template)
from figure_template import display_dataframe, embed_pdf_figure, embed_pdf_pages, std_plot

In [4]:
dataset = 'exorbase'

## Summarize feature stability

### Pi-score

$$\pi = \vert \mathrm{log}_2 FC\vert \cdot (-\mathrm{log}_{10} p_{\mathrm{adj}})$$

### Differential expression

In [5]:
summary = pd.read_table('../output/{}/summary/cross_validation_diffexp/feature_stability.txt'.format(dataset))
summary_subset = summary.query('(fold_change_direction == "any") and (n_features == "10") and (diffexp_method == "deseq2")')
summary_table = summary_subset\
    .groupby(['count_method', 'compare_group'], as_index=True)['feature_stability'].mean()\
    .unstack(level=0)
display_dataframe(
    summary_table,
    filename='summarize_feature_selection',
    format='excel'
)

count_method,featurecounts,featurecounts_lncrna,featurecounts_mrna
compare_group,,,
Normal-CRC,0.862753,0.284193,0.493112
Normal-HCC,0.793598,0.346136,0.726043
Normal-PAAD,0.284300,0.104522,0.361159


In [7]:
summary['count_method'].unique()

array(['featurecounts', 'featurecounts_mrna', 'featurecounts_lncrna'],
      dtype=object)

In [9]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.barplot('count_method', 'feature_stability', hue='compare_group',
            order=['featurecounts', 'featurecounts_mrna', 'featurecounts_lncrna'],
            data=summary_subset,
            errwidth=1.2, capsize=0.05,
            ax=ax,
            )
ax.legend(title='Classifier', bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
ax.set_ylim(0, 1)
ax.set_ylabel('Feature stability')
ax.set_xlabel('Feature type')
ax.set_title('Differential expression')
#std_plot(ax, xlabel='Feature type', ylabel='Feature stability')
fig.tight_layout()
embed_pdf_figure(title='Differential expression feature stability')

### Machine learning

In [22]:
summary = pd.read_table('../output/{}/summary/cross_validation/feature_stability.txt'.format(dataset))
summary_subset = summary.query('(fold_change_direction == "any") \
    and (n_features == "10") \
    and (clustering_score_name == "uca_score")')
summary_table = summary_subset\
    .groupby(['count_method', 'compare_group', 'classifier'], as_index=True)['feature_stability'].mean()\
    .unstack(level=0)
display_dataframe(
    summary_table,
    filename='summarize_feature_selection'
)

count_method                       featurecounts  featurecounts_lncrna  \
compare_group classifier                                                 
Normal-CRC    linear_svm                0.240223              0.337562   
              logistic_regression       0.202443              0.299395   
              random_forest             0.108237              0.364765   
Normal-HCC    linear_svm                0.238015              0.349515   
              logistic_regression       0.233108              0.339376   
              random_forest             0.083786              0.359902   
Normal-PAAD   linear_svm                0.200562              0.389331   
              logistic_regression       0.207349              0.348114   
              random_forest             0.070866              0.145904   

count_method                       featurecounts_mrna  
compare_group classifier                               
Normal-CRC    linear_svm                     0.230315  
              logistic_regression            0.208803  
              random_forest                  0.091755  
Normal-HCC    linear_svm                     0.300904  
              logistic_regression            0.314727  
              random_forest                  0.118420  
Normal-PAAD   linear_svm                     0.153183  
              logistic_regression            0.166106  
              random_forest                  0.111386

In [23]:
for compare_group in summary['compare_group'].unique():
    summary_subset = summary.query('(fold_change_direction == "any") \
    and (n_features == "10") \
    and (clustering_score_name == "uca_score") \
    and (compare_group == "{}")'.format(compare_group))
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot('count_method', 'feature_stability', hue='classifier',
                order=['featurecounts', 'featurecounts_mrna', 'featurecounts_lncrna'],
                data=summary_subset,
                errwidth=1.2, capsize=0.05,
                ax=ax,
                )
    ax.legend(title='Classifier', bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    ax.set_ylim(0, 1)
    ax.set_ylabel('Feature stability')
    ax.set_xlabel('Feature type')
    ax.set_title('{}'.format(compare_group))
    #std_plot(ax, xlabel='Feature type', ylabel='Feature stability')
    fig.tight_layout()
    embed_pdf_figure(title='Feature stability of machine learning ({})'.format(compare_group))

## Metrics on test set

### Differential expression

In [33]:
summary = pd.read_table('../output/{}/summary/cross_validation_diffexp/metrics.test.txt'.format(dataset))
summary_subset = summary.query('(fold_change_direction == "any") and (n_features == "10") and (diffexp_method == "deseq2")')
summary_table = summary_subset \
    .groupby(['count_method', 'compare_group', 'classifier'], as_index=True)['roc_auc'].mean()\
    .unstack(level=0)
display_dataframe(
    summary_table,
    filename='summarize_feature_selection'
)

count_method                       featurecounts  featurecounts_lncrna  \
compare_group classifier                                                 
Normal-CRC    linear_svm                0.748571              0.828571   
              logistic_regression       0.968571              0.928571   
              random_forest             0.979286              0.863571   
Normal-HCC    linear_svm                0.919286              0.875714   
              logistic_regression       0.942857              0.875000   
              random_forest             0.969286              0.838929   
Normal-PAAD   linear_svm                0.754286              0.629524   
              logistic_regression       0.746667              0.651429   
              random_forest             0.673333              0.586190   

count_method                       featurecounts_mrna  
compare_group classifier                               
Normal-CRC    linear_svm                     0.797143  
              logistic_regression            0.805714  
              random_forest                  0.797143  
Normal-HCC    linear_svm                     0.891429  
              logistic_regression            0.931429  
              random_forest                  0.944286  
Normal-PAAD   linear_svm                     0.875238  
              logistic_regression            0.815714  
              random_forest                  0.754286

In [34]:
for compare_group in summary['compare_group'].unique():
    summary_subset = summary.query('(fold_change_direction == "any") \
    and (n_features == 10) \
    and (compare_group == "{}")'.format(compare_group))
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot('count_method', 'roc_auc', hue='classifier',
                order=['featurecounts', 'featurecounts_mrna', 'featurecounts_lncrna'],
                data=summary_subset,
                errwidth=1.2, capsize=0.05,
                ax=ax)
    ax.legend(title='Classifier', bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    ax.set_ylim(0, 1)
    ax.set_ylabel('AUROC')
    ax.set_xlabel('Feature type')
    ax.set_title('{}'.format(compare_group))
    #std_plot(ax, xlabel='Feature type', ylabel='Feature stability')
    fig.tight_layout()
    embed_pdf_figure(title='AUROC of differential expression ({})'.format(compare_group))

### Machine learning

In [31]:
summary = pd.read_table('../output/{}/summary/cross_validation/metrics.test.txt'.format(dataset))
summary_table = summary.query('(fold_change_direction == "any") and (n_features == "10")')\
    .groupby(['count_method', 'compare_group', 'classifier'], as_index=True)['roc_auc'].mean()\
    .unstack(level=0)
display_dataframe(
    summary_table,
    filename='summarize_feature_selection'
)

count_method                       featurecounts  featurecounts_lncrna  \
compare_group classifier                                                 
Normal-CRC    linear_svm                0.964286              0.785714   
              logistic_regression       0.931429              0.802857   
              random_forest             0.990714              0.981429   
Normal-HCC    linear_svm                0.935714              0.877857   
              logistic_regression       0.955714              0.848571   
              random_forest             0.974286              0.982857   
Normal-PAAD   linear_svm                0.838095              0.851429   
              logistic_regression       0.864762              0.773333   
              random_forest             0.936667              0.849524   

count_method                       featurecounts_mrna  
compare_group classifier                               
Normal-CRC    linear_svm                     0.848571  
              logistic_regression            0.857143  
              random_forest                  0.917857  
Normal-HCC    linear_svm                     0.935714  
              logistic_regression            0.953571  
              random_forest                  0.956071  
Normal-PAAD   linear_svm                     0.821905  
              logistic_regression            0.806667  
              random_forest                  0.906667

In [32]:
for compare_group in summary['compare_group'].unique():
    summary_subset = summary.query('(fold_change_direction == "any") \
    and (n_features == 10) \
    and (clustering_score_name == "uca_score") \
    and (compare_group == "{}")'.format(compare_group))
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot('count_method', 'roc_auc', hue='classifier',
                order=['featurecounts', 'featurecounts_mrna', 'featurecounts_lncrna'],
                data=summary_subset,
                errwidth=1.2, capsize=0.05,
                ax=ax)
    ax.legend(title='Classifier', bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    ax.set_ylim(0, 1)
    ax.set_ylabel('AUROC')
    ax.set_xlabel('Feature type')
    ax.set_title('{}'.format(compare_group))
    #std_plot(ax, xlabel='Feature type', ylabel='Feature stability')
    fig.tight_layout()
    embed_pdf_figure(title='AUROC of machine learning ({})'.format(compare_group))